In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics.pairwise import cosine_similarity

In [145]:
file = 'GLCM_local_weights.csv'
csv_weights = np.genfromtxt(file, delimiter=',')

# csv_weights = pd.read_csv('GLCM_local_weights.csv',header=0) #Change the filename for different weights

In [146]:
weight_data = csv_weights

In [147]:
weight_data

array([[-0.05313536, -0.43473392, -0.20840572, -0.00743268, -0.02833411,
        -0.12406313, -0.00860591, -0.17027008, -0.0935675 , -0.19157228,
         0.04905788, -0.08791866, -0.57917438, -0.35802109,  0.27714083],
       [-0.05189695,  0.379825  ,  0.18965523,  0.42577486,  0.41696761,
        -0.03129675, -0.09639582,  0.12075948,  0.33405441,  0.40798327,
        -0.2180343 ,  0.07434648, -0.49487909,  0.97065872,  0.29612557],
       [ 0.20115621, -0.04922264,  0.03267155,  0.20221861, -0.11225222,
        -0.30553163,  0.39652156, -0.12250731, -0.14509387, -0.11866817,
        -0.41899678,  0.39728612, -0.05431928,  0.11136886,  0.05982118],
       [-0.37774392,  0.60481729,  0.03202853,  0.29210187,  0.16162067,
         0.77432104, -0.38216402,  0.73939534, -0.2741613 , -0.02095029,
         0.38143118,  0.7154947 , -1.80333851, -1.3744227 , -0.05965038],
       [-0.10405114, -0.45903298, -0.25775222,  0.04478505,  0.01704786,
        -0.17529442, -0.03965955, -0.16721444, 

In [176]:
cols = [
    'GLCM_Energy_1'
    ,'GLCM_corr_1'
    ,'GLCM_diss_1'
    ,'GLCM_hom_1'
    ,'GLCM_contr_1'
    ,'GLCM_Energy_2'
    ,'GLCM_corr_2'
    ,'GLCM_diss_2'
    ,'GLCM_hom_2'
    ,'GLCM_contr_2'
    ,'GLCM_Energy_3'
    ,'GLCM_corr_3'
    ,'GLCM_diss_3'
    ,'GLCM_hom_3'
    ,'GLCM_contr_3'
    ,'Y'
    # ,'saliency_matrix'
]
excel_data = pd.read_excel('glcm_texture_feature_aug2_cluster8.xlsx', header=0, usecols=range(100))
#data = excel[cols].sample(frac=1).values
data = excel_data[cols].values

# X is all features, y is classified value
X = data[:, 0:-1]
y = data[:, -1]

# Standardizing X
X_std = np.std(X, ddof = 0, axis=0)  # Mean and std on axis=0
X_mean = np.mean(X, axis=0)
X = (X - X_mean) / X_std

no_of_instances = X.shape[0]
no_of_features = X.shape[1]

In [177]:
X

array([[ 2.27093793e-01, -9.16905579e-02,  4.00609024e-01,
         1.06031099e-01,  3.29245057e-01, -4.20162504e-01,
         4.36413695e-02,  1.45547691e+00, -7.38013340e-01,
         1.51601517e+00, -1.16922074e+00, -3.00083452e-01,
         1.35282970e+00, -1.04469376e+00,  1.41587901e+00],
       [-5.21364861e-01, -1.01671532e+00,  8.45895456e-01,
        -7.32119927e-01,  7.59648815e-01,  1.01244857e+00,
        -5.75502551e-01, -1.46915243e-01,  7.06519111e-01,
        -2.88432194e-04, -5.10917223e-01,  7.96082980e-03,
         8.08228771e-01, -9.49544387e-01,  4.31937240e-01],
       [ 3.58839693e-01, -9.16086271e-01,  9.41124691e-02,
         1.77922762e-01,  1.18874208e-01,  1.35815053e+00,
        -1.16877078e+00, -9.97778469e-01,  1.27093968e+00,
        -8.25102367e-01,  3.14591163e-01,  1.20282904e+00,
        -1.71229993e+00,  1.04448285e+00, -1.71083496e+00],
       [-2.41389631e-01, -2.76717938e-01, -9.43105524e-01,
         4.48507431e-02, -1.05643699e+00,  1.29658976

In [179]:
sim_fns = {
    'GLCM_Energy_1'   : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_corr_1'    : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_diss_1'    : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_hom_1'     : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_contr_1'   : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_Energy_2'  : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_corr_2'    : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_diss_2'    : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_hom_2'     : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_contr_2'   : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_Energy_3'  : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_corr_3'    : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_diss_3'    : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_hom_3'     : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    ,'GLCM_contr_3'   : lambda x, y, Xmax, Xmin: abs(x-y)/(Xmax-Xmin)
    }

In [180]:
weight_data.shape

(50, 15)

In [183]:
def predict( train_index, test_index):
    sim_val_arr = []

    for ti_idx, ti in enumerate(train_index):
        sim_val = 0
        
        for idx, Xi in enumerate(X[ti]):
            Ti = X[test_index[0]][idx]
            
            # Maximum & minimum value of the column for the 
            X_max_i = X_max[idx]
            X_min_i = X_min[idx]
            
            attrib = cols[idx]
                        
            # Similarity value from the sim functions multiplied by the weight
            weight_data[ti_idx][idx]
            
            sim_val += sim_fns[attrib](Xi, Ti, X_max_i, X_min_i) * weight_data[ti_idx][idx]
            
        # For each instance append similarity score
        sim_val_arr.append(sim_val)
    
    sim_val_arr = np.array(sim_val_arr)

    nearest_idx = sim_val_arr.argsort()[-1:][::-1]

    k_nearest_solutions = [y[l] for l in nearest_idx]
    #print(k_nearest_match_cases, k_nearest_solutions)
    return k_nearest_solutions

In [184]:
for train_index, test_index in LeaveOneOut().split(X):
    print(train_index, test_index)
    
    predicted = predict(train_index,test_index)
    actual = y[test_index[0]]
    
    print(predicted, actual)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] [0]
[1.0] 0.0
[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] [1]
[1.0] 0.0
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] [2]
[0.0] 0.0
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] [3]
[0.0] 0.0
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] [4]
[1.0] 0.0
[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49] [5]
[0.0] 0.0
[ 0  1  2 

In [185]:
k=1
def test_accuracy():
    correct_total = 0

    # Leave one out cross validation
    for train_index, test_index in LeaveOneOut().split(X):
        predicted = predict(train_index,test_index)
        actual = y[test_index[0]]
        is_correct = 1 if actual in predicted else 0
        print(actual,predicted, is_correct)
        correct_total+=is_correct

    accuracy = correct_total / y.shape[0]
    print("Accuracy for {} neighbors is {}%: ".format(k,accuracy*100))

In [186]:
test_accuracy()

0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [1.0] 0
0.0 [0.0] 1
0.0 [0.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [0.0] 0
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [1.0] 1
1.0 [0.0] 0
0.0 [0.0] 1
0.0 [1.0] 0
Accuracy for 1 neighbors is 52.0%: 
